<a href="https://colab.research.google.com/github/kyoungmin-park/univ-starter/blob/main/%EB%8C%80%ED%95%99%EB%B6%80%EC%8A%A4%ED%83%80%ED%84%B01_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 필요한 라이브러리 불러오기
import os
import pandas as pd
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim
from google.colab import drive

#구글 드라이브 마운트
drive.mount('/content/drive')

# 1. 데이터 경로 설정
train_dir = '/content/drive/Othercomputers/내 노트북/010.연도별 패션 선호도 파악 및 추천 데이터/01-1.정식개방데이터/Training/01.원천데이터'  # 학습 데이터 폴더
val_dir = '/content/drive/Othercomputers/내 노트북/010.연도별 패션 선호도 파악 및 추천 데이터/01-1.정식개방데이터/Validation/01.원천데이터'  # 검증 데이터 폴더

# 2. 학습용 데이터 로드
train_records = []
for filename in os.listdir(train_dir):  # 학습 폴더 내 모든 파일에 대해
    if filename.endswith('.jpg'):  # .jpg 파일만 사용
        parts = filename.split('_')  # 파일명을 '_' 기준으로 나눔
        if len(parts) == 5:  # 파일명이 5부분으로 구성되어 있으면
            img_path = os.path.join(train_dir, filename)  # 전체 경로 구성
            style = parts[3]  # 스타일 정보 추출
            gender = parts[4].replace('.jpg', '')  # 성별 정보 추출 (확장자 제거)
            label = f"{gender}_{style}"  # 라벨 생성 (ex: male_casual)
            train_records.append((img_path, label))  # 경로와 라벨을 리스트에 추가

# DataFrame으로 변환
train_df = pd.DataFrame(train_records, columns=['img_path', 'label'])

# 3. 검증용 데이터 로드 (학습과 동일 방식)
val_records = []
for filename in os.listdir(val_dir):
    if filename.endswith('.jpg'):
        parts = filename.split('_')
        if len(parts) == 5:
            img_path = os.path.join(val_dir, filename)
            style = parts[3]
            gender = parts[4].replace('.jpg', '')
            label = f"{gender}_{style}"
            val_records.append((img_path, label))

val_df = pd.DataFrame(val_records, columns=['img_path', 'label'])

# 4. 라벨 인코딩 (훈련 데이터 기준으로 고정)
label_encoder = LabelEncoder()
train_df['label_idx'] = label_encoder.fit_transform(train_df['label'])  # 훈련 데이터 라벨을 정수로 인코딩
val_df['label_idx'] = label_encoder.transform(val_df['label'])  # 검증 데이터도 같은 인코더로 변환

# 5. 이미지 전처리 정의 (훈련/검증용 다르게 설정)
train_transforms = transforms.Compose([
    transforms.Resize((256, 256)),  # 크기 조정
    transforms.RandomResizedCrop(224),  # 무작위 크롭
    transforms.RandomHorizontalFlip(),  # 좌우 반전
    transforms.ColorJitter(),  # 색상 변화
    transforms.ToTensor(),  # 텐서로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # 정규화
                         std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # 크기 고정
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# 6. PyTorch Dataset 클래스 정의
class FashionDataset(Dataset):
    def __init__(self, dataframe, transform):
        self.df = dataframe.reset_index(drop=True)  # 인덱스 리셋
        self.transform = transform  # 변환 저장

    def __len__(self):
        return len(self.df)  # 데이터 수 반환

    def __getitem__(self, idx):
        img_path = self.df.loc[idx, 'img_path']  # 경로 가져오기
        label = self.df.loc[idx, 'label_idx']  # 라벨 인덱스 가져오기
        image = Image.open(img_path).convert('RGB')  # 이미지 열기 및 RGB로 변환
        image = self.transform(image)  # 변환 적용
        return image, label  # 이미지와 라벨 반환

# 7. DataLoader 생성
train_dataset = FashionDataset(train_df, train_transforms)  # 학습용 데이터셋 생성
val_dataset = FashionDataset(val_df, val_transforms)  # 검증용 데이터셋 생성

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)  # 학습용 로더
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)  # 검증용 로더

# 8. ResNet-18 모델 정의 (사전학습 없이 무작위 초기화)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # GPU 사용 가능 여부 확인

model = models.resnet18(weights=None)  # 사전학습 없이 랜덤 초기화
model.fc = nn.Linear(model.fc.in_features, len(label_encoder.classes_))  # 출력층을 라벨 수에 맞게 수정
model = model.to(device)  # 모델을 디바이스로 이동

# 9. 손실 함수 및 최적화 기법 정의
criterion = nn.CrossEntropyLoss()  # 다중 클래스 분류용 손실 함수
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam 옵티마이저 사용

# 10. 학습 루프
for epoch in range(5):  # 총 5 에폭 동안 학습
    model.train()  # 학습 모드
    total_loss = 0  # 에폭별 손실 초기화
    for images, labels in train_loader:  # 미니배치 반복
        images, labels = images.to(device), labels.to(device)  # 디바이스로 이동
        optimizer.zero_grad()  # 기울기 초기화
        outputs = model(images)  # 예측값 출력
        loss = criterion(outputs, labels)  # 손실 계산
        loss.backward()  # 역전파
        optimizer.step()  # 파라미터 업데이트
        total_loss += loss.item()  # 손실 누적
    print(f"[Epoch {epoch+1}] Training Loss: {total_loss:.4f}")  # 에폭별 손실 출력

# 11. 검증 정확도 평가
model.eval()  # 평가 모드 전환
correct = 0
total = 0
with torch.no_grad():  # 그래디언트 비활성화
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)  # 예측 라벨 추출
        total += labels.size(0)
        correct += (predicted == labels).sum().item()  # 정답 개수 누적

accuracy = correct / total * 100  # 정확도 계산
print(f"✅ Validation Accuracy: {accuracy:.2f}%")  # 정확도 출력

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
